# Dengue Fever - Data Preparation Code

The Dengue Fever dataset can be found on the DrivenData Competition website:
https://www.drivendata.org/competitions/44/dengai-predicting-disease-spread/page/82/

In [1]:
from numpy.random import randn
import numpy as np
import pandas as pd
import seaborn as sns
np.random.seed(123)
import os
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)

# **Step 1: Data Collection**

There are 2 data sets that will be read into a Dataframe:
* dengue_features_train.csv
* dengue_labels_train.csv

In [2]:
# Reading data:
dengue_features_train = pd.read_table('dengue_features_train.csv', sep=',', header=0, delimiter=None, index_col=None, doublequote=True, engine='python')
dengue_features_train.head(5)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [3]:
dengue_labels_train = pd.read_table('dengue_labels_train.csv', sep=',', header=0, delimiter=None, index_col=None, doublequote=True, engine='python')
dengue_labels_train.head(5)

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


# **Step 2: Cleaning Data**

Cleaning Data involves the following key steps:
* 2.a. Understand the data
* 2.b. Handling missing data
* 2.c. Transforming data into meaningful indicators and measures

## 2.a. Understanding the Data

After importing the data, examine the meta data of the data set. 

In [4]:
dengue_features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 24 columns):
city                                     1456 non-null object
year                                     1456 non-null int64
weekofyear                               1456 non-null int64
week_start_date                          1456 non-null object
ndvi_ne                                  1262 non-null float64
ndvi_nw                                  1404 non-null float64
ndvi_se                                  1434 non-null float64
ndvi_sw                                  1434 non-null float64
precipitation_amt_mm                     1443 non-null float64
reanalysis_air_temp_k                    1446 non-null float64
reanalysis_avg_temp_k                    1446 non-null float64
reanalysis_dew_point_temp_k              1446 non-null float64
reanalysis_max_air_temp_k                1446 non-null float64
reanalysis_min_air_temp_k                1446 non-null float64
reanalysis_precip

In [5]:
#Summarizing the number of NaNs present in each column
dengue_features_train.isnull().sum()

city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_mi

**Observations:**
* Total of 1456 records in the data set
* 20 out of the 24 data columns have NaN values. **See Step 2.b. for approaches taken to handle missing data.**
* 'year', 'weekofyear', 'week_start_date' data type will need to be converted to the appropriate time series data type **See Step 2.c. for further details on changing the data type.**

In [6]:
dengue_labels_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 4 columns):
city           1456 non-null object
year           1456 non-null int64
weekofyear     1456 non-null int64
total_cases    1456 non-null int64
dtypes: int64(3), object(1)
memory usage: 45.6+ KB


In [7]:
#Summarizing the number of NaNs present in each column
dengue_labels_train.isnull().sum()

city           0
year           0
weekofyear     0
total_cases    0
dtype: int64

**Observations:**
* Total of 1456 records in the data set.
* No columns have NaN values.

The 'total_cases" data field needs to be joined to the "dengue_features_train" dataset in order to analyze the impacts of the features on the number of Dengue fever cases together.

**The following code was used the join the datasets:**

In [8]:
new_train_set = pd.merge(dengue_features_train,dengue_labels_train, how='left', left_on =['city','year','weekofyear'], right_on=['city','year','weekofyear'])

In [9]:
new_train_set.head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6
5,sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,...,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1,10
9,sj,1990,27,1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,...,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1,6


## 2.b. Handling Missing Data

In the Dengue dataset, 20 out of the 24 data columns contain missing data. Since the environmental attributes for each attribution would specific to the location, filling methods are applied given the City as this will prevent imputing one city's data into another. To do this, the dataset will be split into two different dataframe based on City.¶

In [10]:
df_sj = new_train_set[new_train_set["city"] == "sj"] #dataset for San Juan
df_iq = new_train_set[new_train_set["city"] == "iq"] #dataset for Iquitos

In [11]:
#Extract the column names from the Dataframe to populate into the code
df_sj.columns

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'total_cases'],
      dtype='object')

### 2.b.i. Filling in Missing Data for df_sj (San Juan Dataset)

In [12]:
# Replace nulls in df_sj

# replacing na values with previous value  with limitation of 3 days 
df_sj['reanalysis_air_temp_k'].fillna(method='ffill',limit=3,inplace=True)
df_sj['reanalysis_tdtr_k'].fillna(method='ffill',limit=3,inplace=True)
df_sj['station_diur_temp_rng_c'].fillna(method='ffill',limit=4,inplace=True)
df_sj['ndvi_nw'].fillna(method='ffill',limit=3,inplace=True)
df_sj['ndvi_se'].fillna(method='ffill',limit=3,inplace=True)
df_sj['ndvi_sw'].fillna(method='ffill',limit=3,inplace=True)
df_sj['ndvi_ne'].fillna(method='ffill',limit=3,inplace=True)


# replacing na values with average value
df_sj['precipitation_amt_mm'].fillna(df_sj['precipitation_amt_mm'].mean(),inplace=True)
df_sj['reanalysis_avg_temp_k'].fillna(df_sj['reanalysis_avg_temp_k'].mean(),inplace=True)
df_sj['reanalysis_dew_point_temp_k'].fillna(df_sj['reanalysis_dew_point_temp_k'].mean(),inplace=True)
df_sj['reanalysis_max_air_temp_k'].fillna(df_sj['reanalysis_max_air_temp_k'].mean(),inplace=True)
df_sj['reanalysis_min_air_temp_k'].fillna(df_sj['reanalysis_min_air_temp_k'].mean(),inplace=True)
df_sj['reanalysis_precip_amt_kg_per_m2'].fillna(df_sj['reanalysis_precip_amt_kg_per_m2'].mean(),inplace=True)
df_sj['reanalysis_relative_humidity_percent'].fillna(df_sj['reanalysis_relative_humidity_percent'].mean(),inplace=True)
df_sj['reanalysis_sat_precip_amt_mm'].fillna(df_sj['reanalysis_sat_precip_amt_mm'].mean(),inplace=True)
df_sj['reanalysis_specific_humidity_g_per_kg'].fillna(df_sj['reanalysis_specific_humidity_g_per_kg'].mean(),inplace=True)
df_sj['station_avg_temp_c'].fillna(df_sj['station_avg_temp_c'].mean(),inplace=True)
df_sj['station_min_temp_c'].fillna(df_sj['station_min_temp_c'].mean(),inplace=True)
df_sj['station_max_temp_c'].fillna(df_sj['station_max_temp_c'].mean(),inplace=True)
df_sj['station_precip_mm'].fillna(df_sj['station_precip_mm'].mean(),inplace=True)




# fill nul values with the next value
df_sj['ndvi_nw'].fillna(method='ffill',limit=3,inplace=True)
df_sj['ndvi_se'].fillna(method='ffill',limit=3,inplace=True)
df_sj['ndvi_sw'].fillna(method='ffill',limit=3,inplace=True)
df_sj['ndvi_ne'].fillna(method='ffill',limit=3,inplace=True)

/Users/tammynguyen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [13]:
# check if nulls are replaced
df_sj.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  9
ndvi_nw                                  9
ndvi_se                                  8
ndvi_sw                                  8
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

* There are still NaNs in the dataset. Since the number of NaN records remaining is immaterial, to fill in the remaining missing values, the mean of the respective columns will be used. This should not significantly skew the analysis.

In [14]:
cols_sj=['ndvi_ne','ndvi_nw', 'ndvi_se','ndvi_sw'] #identifying the columns that still have NaN values
df_sj[cols_sj]=df_sj[cols_sj].fillna(df_sj.mean())
df_sj.isnull().sum()

/Users/tammynguyen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

### 2.b.ii. Filling in Missing Data for df_iq (Iquitos Dataset)

In [15]:
# Replace nulls in df_iq

# replacing na values with previeus value  with limitation of 3 days 
df_iq['reanalysis_air_temp_k'].fillna(method='ffill',limit=3,inplace=True)
df_iq['reanalysis_tdtr_k'].fillna(method='ffill',limit=3,inplace=True)
df_iq['station_diur_temp_rng_c'].fillna(method='ffill',limit=4,inplace=True)
df_iq['ndvi_nw'].fillna(method='ffill',limit=3,inplace=True)
df_iq['ndvi_se'].fillna(method='ffill',limit=3,inplace=True)
df_iq['ndvi_sw'].fillna(method='ffill',limit=3,inplace=True)
df_iq['ndvi_ne'].fillna(method='ffill',limit=3,inplace=True)


# replacing na values with average value
df_iq['reanalysis_avg_temp_k'].fillna(df_iq['reanalysis_avg_temp_k'].mean(),inplace=True)
df_iq['reanalysis_dew_point_temp_k'].fillna(df_iq['reanalysis_dew_point_temp_k'].mean(),inplace=True)
df_iq['reanalysis_max_air_temp_k'].fillna(df_iq['reanalysis_max_air_temp_k'].mean(),inplace=True)
df_iq['reanalysis_min_air_temp_k'].fillna(df_iq['reanalysis_min_air_temp_k'].mean(),inplace=True)
df_iq['reanalysis_precip_amt_kg_per_m2'].fillna(df_iq['reanalysis_precip_amt_kg_per_m2'].mean(),inplace=True)
df_iq['reanalysis_specific_humidity_g_per_kg'].fillna(df_iq['reanalysis_specific_humidity_g_per_kg'].mean(),inplace=True)
df_iq['reanalysis_sat_precip_amt_mm'].fillna(df_iq['reanalysis_sat_precip_amt_mm'].mean(),inplace=True)
df_iq['reanalysis_relative_humidity_percent'].fillna(df_iq['reanalysis_relative_humidity_percent'].mean(),inplace=True)
df_iq['station_avg_temp_c'].fillna(df_iq['station_avg_temp_c'].mean(),inplace=True)
df_iq['station_min_temp_c'].fillna(df_iq['station_min_temp_c'].mean(),inplace=True)
df_iq['station_max_temp_c'].fillna(df_iq['station_max_temp_c'].mean(),inplace=True)
df_iq['station_precip_mm'].fillna(df_iq['station_precip_mm'].mean(),inplace=True)
df_iq['precipitation_amt_mm'].fillna(df_iq['precipitation_amt_mm'].mean(),inplace=True)


# fill nul values with the next value
df_iq['ndvi_nw'].fillna(method='ffill',limit=3,inplace=True)
df_iq['ndvi_se'].fillna(method='ffill',limit=3,inplace=True)
df_iq['ndvi_sw'].fillna(method='ffill',limit=3,inplace=True)
df_iq['ndvi_ne'].fillna(method='ffill',limit=3,inplace=True)

In [16]:
# check if nulls are replaced
df_iq.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  3
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

* There are still NaNs in the dataset. Since the number of NaN records remaining is immaterial, to fill in the remaining missing values, the mean of the respective columns will be used. This should not significantly skew the analysis.

In [17]:
cols_iq=['reanalysis_sat_precip_amt_mm','station_diur_temp_rng_c'] #identifying the columns that still have NaN values
df_iq[cols_iq]=df_iq[cols_iq].fillna(df_iq.mean())
df_iq.isnull().sum()

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

## 2.c. Transforming Data
1. Converting time series data types
2. Create New Data Fields
    * a) Vegetation

### (1) Converting "week_start_date" to DateObject

In [18]:
#Convert week_start_date into a date object
df_sj['week_start_date']=pd.to_datetime(df_sj['week_start_date'])
df_iq['week_start_date']=pd.to_datetime(df_iq['week_start_date'])

/Users/tammynguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/tammynguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df_sj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 936 entries, 0 to 935
Data columns (total 25 columns):
city                                     936 non-null object
year                                     936 non-null int64
weekofyear                               936 non-null int64
week_start_date                          936 non-null datetime64[ns]
ndvi_ne                                  936 non-null float64
ndvi_nw                                  936 non-null float64
ndvi_se                                  936 non-null float64
ndvi_sw                                  936 non-null float64
precipitation_amt_mm                     936 non-null float64
reanalysis_air_temp_k                    936 non-null float64
reanalysis_avg_temp_k                    936 non-null float64
reanalysis_dew_point_temp_k              936 non-null float64
reanalysis_max_air_temp_k                936 non-null float64
reanalysis_min_air_temp_k                936 non-null float64
reanalysis_precip_amt_kg_

In [20]:
df_iq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 936 to 1455
Data columns (total 25 columns):
city                                     520 non-null object
year                                     520 non-null int64
weekofyear                               520 non-null int64
week_start_date                          520 non-null datetime64[ns]
ndvi_ne                                  520 non-null float64
ndvi_nw                                  520 non-null float64
ndvi_se                                  520 non-null float64
ndvi_sw                                  520 non-null float64
precipitation_amt_mm                     520 non-null float64
reanalysis_air_temp_k                    520 non-null float64
reanalysis_avg_temp_k                    520 non-null float64
reanalysis_dew_point_temp_k              520 non-null float64
reanalysis_max_air_temp_k                520 non-null float64
reanalysis_min_air_temp_k                520 non-null float64
reanalysis_precip_amt_

### (2) Creating New Data Fields

#### (2) a. Vegetation Type

The NDVI values individuate the type of vegetation in a given region:
1.	Water - Negative values approaching -1
2.	Barren areas of rock, sand, or snow - Values close to zero (-0.1 to 0.1)
3.	Shrub/Grassland - Low, positive values (approximately 0.2 to 0.4)
4.	Tropical Rainforest - High values (values approaching 1).

The above rules above was used to assign the vegation type to the NDVI values for 'ndvi_ne','ndvi_nw','ndvi_se' and 'ndvi_sw'.

In [21]:
#Step 1: Merge the data set back together
df_merge=pd.concat([df_sj,df_iq])
df_merge.head(5)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [22]:
#Assignment the df_merge type to the NDVI value
df_merge['veg_type_ne']=df_merge['ndvi_ne'].apply(lambda x:'water' if x<-0.1
                                              else 'barren' if (x<=0.1 and x>=-0.1)
                                              else 'grassland' if (x<=0.4 and x>0.1)
                                              else 'tropical' if (x<=1.0 and x>0.4)
                                              else 'unknown')
df_merge['veg_type_nw']=df_merge['ndvi_nw'].apply(lambda x:'water' if x<-0.1
                                              else 'barren' if (x<=0.1 and x>=-0.1)
                                              else 'grassland' if (x<=0.4 and x>0.1)
                                              else 'tropical' if (x<=1.0 and x>0.4)
                                              else 'unknown')
df_merge['veg_type_se']=df_merge['ndvi_se'].apply(lambda x:'water' if x<-0.1
                                              else 'barren' if (x<=0.1 and x>=-0.1)
                                              else 'grassland' if (x<=0.4 and x>0.1)
                                              else 'tropical' if (x<=1.0 and x>0.4)
                                              else 'unknown')
df_merge['veg_type_sw']=df_merge['ndvi_sw'].apply(lambda x:'water' if x<-0.1
                                              else 'barren' if (x<=0.1 and x>=-0.1)
                                              else 'grassland' if (x<=0.4 and x>0.1)
                                              else 'tropical' if (x<=1.0 and x>0.4)
                                              else 'unknown')
df_merge.head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,veg_type_ne,veg_type_nw,veg_type_se,veg_type_sw
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,25.442857,6.900000,29.4,20.0,16.0,4,grassland,grassland,grassland,grassland
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,26.714286,6.371429,31.7,22.2,8.6,5,grassland,grassland,grassland,grassland
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.714286,6.485714,32.2,22.8,41.4,4,barren,grassland,grassland,grassland
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,27.471429,6.771429,33.3,23.3,4.0,3,grassland,grassland,grassland,grassland
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,28.942857,9.371429,35.0,23.9,5.8,6,grassland,grassland,grassland,grassland
5,sj,1990,23,1990-06-04,0.196200,0.174850,0.254314,0.181743,9.58,299.630000,...,28.114286,6.942857,34.4,23.9,39.1,2,grassland,grassland,grassland,grassland
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,27.414286,6.771429,32.2,23.3,29.7,4,grassland,barren,grassland,grassland
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,28.371429,7.685714,33.9,22.8,21.1,5,barren,barren,grassland,grassland
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,28.328571,7.385714,33.9,22.8,21.1,10,grassland,grassland,grassland,grassland
9,sj,1990,27,1990-07-02,0.102450,0.121550,0.160683,0.202567,14.41,300.154286,...,28.328571,6.514286,33.9,24.4,1.1,6,grassland,grassland,grassland,grassland


The "df_sj" and "df_iq" DataFrames are then saved as 2 separate csv files.

In [23]:
df_sj.to_csv('df_sj.csv')
df_iq.to_csv('df_iq.csv')